In [1]:
import math
import json
import requests
import datetime as dt
import numpy as np
import xml.etree.ElementTree as ET
from scipy import optimize
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
from matplotlib import pyplot as plt

In [2]:
def get_trsy_data(trsy_year):
    trsy_url = f'https://home.treasury.gov/resource-center/data-chart-center/interest-rates/pages/xml?data=daily_treasury_yield_curve&field_tdr_date_value={trsy_year}'
    print(f'fetching treasury par rates: {trsy_url}')
    trsy_response = requests.get(trsy_url)
    if trsy_response.status_code!=200:
        print(f'error: {trsy_response.status_code} - {trsy_response.content}')
        return
    raw_str = trsy_response.content.decode('utf=8')
    ns_dict = {'main': raw_str.split('xmlns="')[1].split('"')[0],
               'm': raw_str.split('xmlns:m="')[1].split('"')[0],
               'd': raw_str.split('xmlns:d="')[1].split('"')[0]}
    year_fracs = {'BC_1MONTH':0.0833,'BC_2MONTH':0.1666,'BC_3MONTH':0.25,'BC_4MONTH':0.3333,'BC_6MONTH':0.5,'BC_1YEAR':1.0, 
                  'BC_2YEAR':2.0,'BC_3YEAR':3.0,'BC_5YEAR':5.0,'BC_7YEAR':7.0,'BC_10YEAR':10.0,'BC_20YEAR':20.0,'BC_30YEAR':30.0}
    root = ET.fromstring(raw_str)
    parsed_data = {}
    for entry in root.findall('main:entry', ns_dict):
        extract = dict([(raw_dp.tag.replace('{'+ns_dict['d']+'}',''), raw_dp.text) for raw_dp in entry.find('main:content/m:properties', ns_dict)])
        parsed_data[dt.datetime.strptime(extract['NEW_DATE'].split('T')[0],'%Y-%m-%d').date()] = dict([(year_fracs[k],float(v)/100) for k,v in extract.items() if k in year_fracs])
    return parsed_data

def get_rate(t, param): # Nelson-Siegel-Svensson : https://en.wikipedia.org/wiki/Fixed-income_attribution#Modeling_the_yield_curve
    b0,b1,b2,b3,d0,d1 = param
    x = t/d0
    x_ = t/d1
    y = -math.exp(-x)
    y_ = -math.exp(-x_)
    z = (1+y)/x
    return b0+b1*z+b2*(z+y)+b3*((1+y_)/x_+y_)

def fit_curve(known, guess=[0.02,0.03,0.04,0.05,0.7,13.6]):
    errors = lambda params, known: sum([(get_rate(k,params)-v)**2 for k,v in known.items()])
    res = optimize.minimize(errors, guess, args=(known))
    if res['success']:
        return res['x']
    
def bootstrap_spot(par_params):
    times = np.linspace(0.5,30,60)
    spots = []
    while len(spots)<len(times):
        c = max(0,get_rate(times[len(spots)], par_params)/2)
        spots.append(((1+c)/(1-sum([c/((1+s)**t) for s,t in zip(spots,times)])))**(1/times[len(spots)])-1) # cheeky
    return dict(zip(times,spots))

def calculate_params(as_of, par_rates):
    print(f'rates as of : {as_of}')
    init_guess = [0.02,0.03,0.04,0.05,0.74,13.67]
    print('fitting model to par curve')
    curve_params = {'par': fit_curve(par_rates, guess=init_guess)}
    print('bootstrapping spot curve')
    spot_bootstrap = bootstrap_spot(curve_params['par'])
    print('fitting model to spot curve')
    curve_params['spot'] = fit_curve(spot_bootstrap, guess=init_guess)
    return curve_params


all_par_data = get_trsy_data(2023)
latest_data_date = list(all_par_data.keys())[-1]
latest_par_rates = all_par_data[latest_data_date]
model_params = calculate_params(latest_data_date, latest_par_rates)



fetching treasury par rates: https://home.treasury.gov/resource-center/data-chart-center/interest-rates/pages/xml?data=daily_treasury_yield_curve&field_tdr_date_value=2023


rates as of : 2023-12-29
fitting model to par curve
bootstrapping spot curve
fitting model to spot curve


In [4]:
def get_fwd_rate(fwd, term, spot_params):
    term = max(term, 0.001)
    a = (get_rate(fwd + term, spot_params) + 1)**(fwd + term)
    b = (get_rate(fwd, spot_params) + 1)**(fwd)
    return (a/b)**(1/term) - 1

def price_bond(sim_date, bond, yld):
    if sim_date >= bond[0]:
        fwd = (bond[0]-dt.date.today()).days/365
        term = (sim_date-bond[0]).days/365
        rate = get_fwd_rate(fwd,term,model_params['spot'])
        return 100*(1+rate)**term
    n,r = divmod((bond[0]-sim_date).days,183)
    return sum([(bond[2]*50)/(1+yld)**((r/365)+(t*0.5)) for t in range(n)]+[(bond[2]*50+100)/(1+yld)**(r/365+n*0.5)])

class GUI:
    def __init__(self, params, portfolio, ladder, BH=False):
        self.params = params
        self.port = portfolio
        self.ladder = ladder
        self.do_BH = BH
        par_graph_on = widgets.ToggleButton(value=True, description='Trsy - Par')
        spot_graph_on = widgets.ToggleButton(value=False, description='Trsy - Spot')
        fwd_graph_on = widgets.ToggleButton(value=False, description='Trsy - Fwd')
        oas_graph_on = widgets.ToggleButton(value=False, description='Corp - Trsy+OAS')
        ladder_graph_on = widgets.ToggleButton(value=False, description='Txbl Muni Ladder')
        self.forward_slider = widgets.FloatSlider(value=0, min=0.005, max=5.15, description='Forward Time (yrs):', continuous_update=True, style=dict(description_width='initial'), layout=Layout(width='auto'))
        trsy_level = widgets.FloatSlider(value=0, min=-0.05, max=0.05, step=0.0005, description='Adjust Trsy Level:', continuous_update=True, style=dict(description_width='initial'), layout=Layout(width='auto'))
        self.account_box = widgets.VBox([widgets.ToggleButton(value=False, description=acct) for acct in self.port.keys()])
        resolution_slider = widgets.IntSlider(value=30, min=20, max=100, description='Resolution:', continuous_update=False, style=dict(description_width='initial'), layout=Layout(width='auto'))
        x_range_slider = widgets.FloatRangeSlider(value=[0.01,10], min=0.01, max=30, step=0.08, description='x-axis bounds (yrs):', continuous_update=False, style=dict(description_width='initial'), layout=Layout(width='auto'))
        y_range_slider = widgets.FloatRangeSlider(value=[0.03,0.075], min=0.0, max=0.1, step=0.005, description='y-axis bounds (%):', continuous_update=False, style=dict(description_width='initial'), layout=Layout(width='auto'))

        self.ladder['stats'] = {'curr_mv':sum([bond[3]*price_bond(dt.date.today(), bond, bond[1]) for bond in self.ladder['bonds']]) if not self.do_BH else 493253.10}
        self.ladder_widgets = {'sim_val':widgets.Text(description='Simulated Value', value=f'${self.ladder["stats"]["curr_mv"]:,.2f}',style=dict(description_width='100px'), layout=Layout(width='300px')),
                               'gainloss': widgets.Text(description='GAIN/LOSS', value=f'${0:,.2f}', style=dict(description_width='100px'), layout=Layout(width='300px')),
                               'annl_ret': widgets.Text(description='Ann. Gross Ret.',value=f'{0:.2%}', style=dict(description_width='100px'), layout=Layout(width='300px')),
                               'annl_ret_net': widgets.Text(description='Ann. NET Return',value=f'{0:.2%}', style=dict(description_width='100px'), layout=Layout(width='300px'))}

        settings_panel = widgets.VBox([widgets.Label('Plotting Yield Curves:'), 
                                       widgets.HBox([par_graph_on, spot_graph_on, fwd_graph_on]), 
                                       oas_graph_on,
                                       widgets.Label('Simulation Settings:', layout=Layout(border_top='1px dashed black')),
                                       self.forward_slider,
                                       trsy_level,
                                       widgets.Label('Portfolio/Account Settings:', layout=Layout(border_top='1px dashed black')),
                                       self.account_box,
                                       widgets.Label('Graph Settings:', layout=Layout(border_top='1px dashed black')),
                                       resolution_slider, 
                                       x_range_slider, 
                                       y_range_slider,
                                       widgets.Label('Ladder Portfolio:', layout=Layout(border_top='1px dashed black')),
                                       ladder_graph_on,
                                       self.ladder_widgets['sim_val'],
                                       self.ladder_widgets['gainloss'],
                                       self.ladder_widgets['annl_ret'],
                                       self.ladder_widgets['annl_ret_net'],
                                       ], layout=Layout(width='100%', border='1px solid black'))
       

        for acct in self.port.keys():
            self.port[acct]['stats'] = {'curr_mv':sum([bond[3]*price_bond(dt.date.today(), bond, bond[1]) for bond in self.port[acct]['bonds']])}
        self.stats_widgets = {acct:{'name': widgets.Label(acct),
                                    'start_val':widgets.Text(description='Current Value', value=f'${self.port[acct]["stats"]["curr_mv"]:,.2f}',style=dict(description_width='100px'), layout=Layout(width='300px')),
                                    'sim_val': widgets.Text(description='Simulated Value', value=f'${self.port[acct]["stats"]["curr_mv"]:,.2f}', style=dict(description_width='100px'), layout=Layout(width='300px')),
                                    'price_ret': widgets.Text(description='Price Return', value=f'{0:.2%}', style=dict(description_width='100px'), layout=Layout(width='300px')),
                                    'price_ret_ann': widgets.Text(description='PR (annl.)',value=f'{0:.2%}', style=dict(description_width='100px'), layout=Layout(width='300px')),
                                    'income_ret': widgets.Text(description='Income Return',value=f'{0:.2%}', style=dict(description_width='100px'), layout=Layout(width='300px')),
                                    'income_ret_ann': widgets.Text(description='Income (annl.)',value=f'{0:.2%}', style=dict(description_width='100px'), layout=Layout(width='300px')),
                                    } for acct in self.port.keys()}
        self.stats_widgets['total'] = {'name': widgets.Label('TOTAL PORTFOLIO'),
                                       'gainloss': widgets.Text(description='GAIN/LOSS', value=f'${0:,.2f}', style=dict(description_width='100px'), layout=Layout(width='300px')),
                                       'annl_ret': widgets.Text(description='Ann. Gross Return',value=f'{0:.2%}', style=dict(description_width='100px'), layout=Layout(width='300px'))}
        stats_panel = widgets.VBox([widgets.VBox([j for j in self.stats_widgets[i].values()]) for i in self.stats_widgets], layout=Layout(width='100%', border='1px solid black'))
        graph_output = widgets.interactive_output(self.graph_handler, {'plotting_par': par_graph_on,
                                                                       'plotting_spot': spot_graph_on,
                                                                       'plotting_fwd': fwd_graph_on,
                                                                       'plotting_oas': oas_graph_on,
                                                                       'plotting_ladder': ladder_graph_on,
                                                                       'fwd_time': self.forward_slider,
                                                                       'trsy_level': trsy_level,
                                                                       'resolution': resolution_slider,
                                                                       'x_range': x_range_slider, 
                                                                       'y_range': y_range_slider} | {a.description:a for a in self.account_box.children})
        graph_panel = widgets.VBox([graph_output], layout=Layout(width='100%', border='1px solid black'))
        self.main = widgets.AppLayout(left_sidebar=settings_panel, center=graph_panel, right_sidebar=stats_panel, pane_widths=[2, 5, 2])
        display(self.main)
        return
    
    def graph_handler(self, plotting_par, plotting_spot, plotting_fwd, plotting_oas, plotting_ladder, fwd_time, trsy_level, resolution, x_range, y_range, **kwargs):
        fig, ax = plt.subplots(figsize=(12,8))
        plot_times = np.linspace(x_range[0], x_range[1],resolution)
        if plotting_par:
            par_rates = [get_rate(t, self.params['par']) for t in plot_times]
            ax.plot(plot_times,par_rates,'m')
        if plotting_spot:
            spot_rates = [get_rate(t, self.params['spot']) for t in plot_times]
            ax.plot(plot_times,spot_rates, 'r')
        if plotting_fwd:
            self.forward_slider.disabled = False
            fwd_times = np.linspace(x_range[0], min(30-fwd_time,x_range[1]), resolution)
            fwd_rates = [get_fwd_rate(fwd_time, t, self.params['spot']) for t in fwd_times]
            fwd_rates_adj = [r + trsy_level for r in fwd_rates]
            ax.plot(fwd_times,fwd_rates,'b--')
            ax.plot(fwd_times,fwd_rates_adj, 'b')
        else:
            self.forward_slider.disabled = True
        if plotting_oas:
            if plotting_fwd:
                oas_rates = [get_rate(t, self.params['oas']) + r for t,r in zip(fwd_times, fwd_rates)]
                oas_rates_adj = [r + trsy_level for r in oas_rates]
                ax.plot(fwd_times,oas_rates,'g--')
                ax.plot(fwd_times,oas_rates_adj, 'g')
            elif plotting_spot:
                oas_rates = [get_rate(t, self.params['oas']) + r for t,r in zip(plot_times, spot_rates)]
                ax.plot(plot_times,oas_rates, 'g')

        sim_date = dt.date.today() + dt.timedelta(days=fwd_time*365)
        if plotting_ladder:
            ladder_bond_times = [(b[0] - sim_date).days/365 for b in self.ladder['bonds']]
            if plotting_fwd:
                ladder_rates = [get_rate(t, self.params['tmuni']) + r for t,r in zip(fwd_times, fwd_rates)]
                ladder_rates_adj = [r + trsy_level for r in ladder_rates]
                if self.do_BH:
                    ladder_bond_rates = [b[4]*(get_fwd_rate(fwd_time,t,self.params['spot'])+trsy_level) + \
                                         (1-b[4])*(get_fwd_rate(fwd_time,t,self.params['spot'])+trsy_level+get_rate(t, self.params['tmuni'])) for t,b in zip(ladder_bond_times, self.ladder['bonds'])]
                else:
                    ladder_bond_rates = [get_rate(t,self.params['tmuni'])+get_fwd_rate(fwd_time, t, self.params['spot'])+trsy_level for t in ladder_bond_times]
                ax.plot(fwd_times,ladder_rates,'c--')
                ax.plot(fwd_times,ladder_rates_adj, 'c')
                ax.scatter(ladder_bond_times, ladder_bond_rates)
                self.update_ladder_stats(sim_date,trsy_level)
            elif plotting_spot:
                ladder_rates = [get_rate(t, self.params['tmuni']) + r for t,r in zip(plot_times, spot_rates)]
                ax.plot(plot_times,ladder_rates, 'c')
            
        total_gainloss = 0
        for acct in kwargs:
            if kwargs[acct]:
                bond_times = [(b[0] - sim_date).days/365 for b in self.port[acct]['bonds']]
                bond_rates = [b[4]*(get_fwd_rate(fwd_time,t,self.params['spot'])+trsy_level) + \
                              (1-b[4])*(get_fwd_rate(fwd_time,t,self.params['spot'])+trsy_level+get_rate(t, self.params['oas'])) for t,b in zip(bond_times, self.port[acct]['bonds'])]
                ax.scatter(bond_times,bond_rates)
                total_gainloss += self.update_stats(acct,bond_rates,sim_date)
        self.update_portfolio_stats(total_gainloss, sim_date)
        ax.set_xlim(x_range)
        ax.set_ylim(y_range)
        plt.show()
        return
    
    def update_stats(self,acct,bond_rates,sim_date):
        sim_mv = sum([bond[3]*price_bond(sim_date,bond,yld) for bond,yld in zip(self.port[acct]['bonds'],bond_rates)])
        dollar_income = sum([sum([bond[2]*50*bond[3] for i in [(bond[0]-sim_date).days%183 + n*183 for n in range(1+math.floor((bond[0]-dt.date.today()).days/183))] 
                               if i<=(sim_date-dt.date.today()).days]) for bond in self.port[acct]['bonds']])
        t = ((sim_date-dt.date.today()).days/365)
        price_ret = (sim_mv/self.port[acct]['stats']['curr_mv'])-1
        pr_annl = (1+price_ret)**(1/t)-1 if t>1 else price_ret
        income_ret = dollar_income/self.port[acct]['stats']['curr_mv']
        ir_annl = (1+income_ret)**(1/t)-1 if t>1 else income_ret
        self.stats_widgets[acct]['sim_val'].value = f'${sim_mv:,.2f}'
        self.stats_widgets[acct]['price_ret'].value = f'{price_ret:.2%}'
        self.stats_widgets[acct]['price_ret_ann'].value = f'{pr_annl:.2%}'
        self.stats_widgets[acct]['income_ret'].value = f'{income_ret:.2%}'
        self.stats_widgets[acct]['income_ret_ann'].value = f'{ir_annl:.2%}'
        return (sim_mv-self.port[acct]['stats']['curr_mv'])+dollar_income
    
    def update_portfolio_stats(self,gl,sim_date):
        t = ((sim_date-dt.date.today()).days/365)
        self.stats_widgets['total']['gainloss'].value = f'${gl:,.2f}'
        self.stats_widgets['total']['annl_ret'].value = '{:.2%}'.format((1+gl/sum([self.port[acct]['stats']['curr_mv'] for acct in self.port]))**(1/t if t>1 else 1)-1)
        return
    
    def update_ladder_stats(self,sim_date,trsy_level):
        t = ((sim_date-dt.date.today()).days/365)
        bond_income = lambda start,mat,sim_date,rate: sum([rate*50 for j in [(i*183)+((mat-start).days%183) for i in range(math.floor((mat-start).days/183))] if j<=(sim_date-start).days])

        ladder_income = 0
        fix_bonds = self.ladder['bonds']
        for bond in self.ladder['bonds']:
            if sim_date>bond[0]:
                new_bond_mat = bond[0]+dt.timedelta(days=365*12)
                new_bond_rate = get_fwd_rate(((bond[0]-dt.date.today()).days/365),12,self.params['spot']) + get_rate(t,self.params['tmuni']) + trsy_level
                new_bond_data = (new_bond_mat, new_bond_rate, new_bond_rate, bond[3], 0)
                ladder_income += bond[3]*bond_income(bond[0], new_bond_mat, sim_date, new_bond_rate)
                fix_bonds = [b for b in fix_bonds if b!=bond]
                fix_bonds.append(new_bond_data)
            elif bond[0] > sim_date+dt.timedelta(days=365*12):
                old_bond_mat = bond[0]-dt.timedelta(days=365*12)
                old_bond_rate = get_rate((old_bond_mat-dt.date.today()).days/365,self.params['spot']) + get_rate(t,self.params['tmuni']) + trsy_level
                old_bond_data = (old_bond_mat, old_bond_rate, old_bond_rate, bond[3], 0)
                ladder_income += bond[3]*bond_income(dt.date.today(), old_bond_mat, sim_date, old_bond_rate)
                fix_bonds = [b for b in fix_bonds if b!=bond]
                fix_bonds.append(old_bond_data)
            else:
                ladder_income += bond[3]*bond_income(dt.date.today(), bond[0], sim_date, bond[2])
        self.ladder['bonds'] = fix_bonds

        ladder_bond_times = [(b[0] - sim_date).days/365 for b in self.ladder['bonds']]
        ladder_bond_rates = [get_rate(t,self.params['tmuni'])+get_fwd_rate(((sim_date-dt.date.today()).days/365), t, self.params['spot'])+trsy_level for t in ladder_bond_times]


        sim_value = sum([bond[3]*price_bond(sim_date,bond,yld) for bond,yld in zip(self.ladder['bonds'], ladder_bond_rates)])
        gl = sim_value - self.ladder['stats']['curr_mv']
        tot_ret = (1+(gl+ladder_income)/self.ladder['stats']['curr_mv'])**(1/t if t>1 else 1)-1
        self.ladder_widgets['sim_val'].value = f'${sim_value:,.2f}'
        self.ladder_widgets['gainloss'].value = f'${gl+ladder_income:,.2f}'
        self.ladder_widgets['annl_ret'].value = f'{tot_ret:.2%}'
        #self.ladder_widgets['annl_ret_net'].value = '{:.2%}'.format(tot_ret-((1.01)**(1/t if t>1 else 1)-1))
        self.ladder_widgets['annl_ret_net'].value = '{:.2%}'.format((1+(gl+ladder_income)/self.ladder['stats']['curr_mv']-t*0.01)**(1/t if t>1 else 1)-1)


        return

# MANUAL - GET CURRENT CORPORATE BOND OPTION-ADJUSTED SPREADS...
# go to https://indices.ice.com/
# record {'Maturity/WAL':'OAS-Govt'} for the following AAA-A indices:
oas_data = {0.515:0.0059, # C01A
            1.940:0.0080, # C110
            2.783:0.0090, # CV10
            3.950:0.0104, # C210
            4.412:0.0101, # C510
            6.025:0.0109, # C310
            7.450:0.0122, # C610
            8.558:0.0133, # C410
            12.86:0.0116, # C710
            23.02:0.0108, # C910
            24.78:0.0107} # C810
taxmuni_data = {0.515:0.0042,
                1.783:0.0038, # D1TM
                3.973:0.0063, # D2TM
                5.920:0.0080, # D3TM
                9.066:0.0125, # D4TM
                6.519:0.0092, # D5TM
                7.903:0.0108, # D6TM
                11.93:0.0106, # D7TM
                24.37:0.0096, # D8TM
                22.13:0.0098} # D9TM
model_params['oas'] = fit_curve(oas_data)
model_params['tmuni'] = fit_curve(taxmuni_data) #np.array([0.052734, 0.234037, 0.0, 0.244048, 0.026877, 192.0786]) #fit_curve(taxmuni_data)

relative_spread = lambda mat,yld,series:(lambda t: -(yld-(get_rate(t,model_params[series])+get_rate(t,model_params['spot'])))/(get_rate(t,model_params[series])))((mat-dt.date.today()).days/365)
portfolio = {'account1': {'bonds':[(dt.date(2027,2,28), 0.0503, 0.01125, 200), (dt.date(2026,8,31), 0.051, 0.01375, 250), (dt.date(2027,3,26), 0.0565, 0.033, 100), (dt.date(2027,3,25), 0.0581, 0.0375, 150),
                                              (dt.date(2026,9,11), 0.054, 0.0205, 150), (dt.date(2028,12,6), 0.0558, 0.039, 150), (dt.date(2028,4,11), 0.0547, 0.037, 150), (dt.date(2027,8,22), 0.0556, 0.0315, 150),
                                                (dt.date(2026,11,1), 0.0552, 0.025, 150)]},
             'account2': {'bonds':[(dt.date(2027,2,28), 0.0503, 0.01125, 400), (dt.date(2026,8,31), 0.051, 0.01375, 450), (dt.date(2027,3,26), 0.0565, 0.033, 250), (dt.date(2027,3,25), 0.0581, 0.0375, 250),
                                               (dt.date(2026,9,11), 0.054, 0.0205, 250), (dt.date(2028,12,6), 0.0558, 0.039, 250), (dt.date(2028,4,11), 0.0547, 0.037, 250), (dt.date(2027,8,22), 0.0556, 0.0315, 250), 
                                               (dt.date(2026,11,1), 0.0552, 0.025, 250)]},
             'account3': {'bonds':[(dt.date(2027,2,28), 0.0503, 0.01125, 200), (dt.date(2026,8,31), 0.051, 0.01375, 250), (dt.date(2027,3,25), 0.0581, 0.0375, 150), (dt.date(2026,9,11), 0.054, 0.0205, 150), 
                                         (dt.date(2028,12,6), 0.0558, 0.039, 150), (dt.date(2028,4,11), 0.0547, 0.037, 150), (dt.date(2027,8,22), 0.0556, 0.0315, 150), (dt.date(2026,11,1), 0.0552, 0.025, 150)]}}
portfolio = {acct:{'bonds':[(*b, relative_spread(b[0], b[1], 'oas')) for b in portfolio[acct]['bonds']]} for acct in portfolio.keys()}

ladder = {'bonds':[(dt.date(2024,10,19), 0.0579, 0.05794, 410), (dt.date(2025,10,19), 0.0564, 0.0564, 410), (dt.date(2026,10,19), 0.0567, 0.0567, 411), (dt.date(2027,10,19), 0.0574, 0.0574, 411), 
                  (dt.date(2028,10,19), 0.058, 0.058, 410), (dt.date(2029,10,19), 0.0586, 0.0586, 411), (dt.date(2030,10,19), 0.0592, 0.05919, 411), (dt.date(2031,10,19), 0.0596, 0.0596, 410), 
                  (dt.date(2032,10,19), 0.0601, 0.0601, 410), (dt.date(2033,10,19), 0.0605, 0.0605, 410), (dt.date(2034,10,19), 0.0609, 0.06086, 410), (dt.date(2035,10,19), 0.0611, 0.0611, 410)]}

# ladder = {'bonds':[(dt.date(2024,4,19), 0.0752, 0.052, 200), (dt.date(2028,11,15), 0.0629, 0.0532, 200), (dt.date(2026,6,1), 0.0554, 0.0108, 102), (dt.date(2029,2,1), 0.0602, 0.05, 205), 
#                    (dt.date(2026,11,1), 0.0408, 0.04, 230), (dt.date(2033,6,1), 0.06, 0.02048, 102), (dt.date(2026,12,1), 0.0556, 0.0175, 102), (dt.date(2030,8,1), 0.0554, 0.02013, 256), 
#                    (dt.date(2029,8,15), 0.063, 0.0242, 102), (dt.date(2030,11,15), 0.057, 0.03923, 205), (dt.date(2025,6,1), 0.0599, 0.0315, 205), (dt.date(2025,4,1), 0.0624, 0.04411, 154), 
#                    (dt.date(2031,11,1), 0.0594, 0.02772, 102), (dt.date(2028,8,1), 0.0561, 0.02552, 205), (dt.date(2031,8,1), 0.0569, 0.02587, 205), (dt.date(2025,11,15), 0.0603, 0.02539, 256), 
#                    (dt.date(2024,8,1), 0.0628, 0.01214, 205), (dt.date(2028,11,1), 0.0491, 0.035, 205), (dt.date(2027,8,1), 0.0561, 0.03278, 102), (dt.date(2027,4,1), 0.0575, 0.03538, 205), 
#                    (dt.date(2035,12,1), 0.0631, 0.05946, 102), (dt.date(2033,8,1), 0.058, 0.02119, 154), (dt.date(2029,8,15), 0.0556, 0.02, 102), (dt.date(2030,3,1), 0.0562, 0.02742, 154), 
#                    (dt.date(2024,4,1), 0.0614, 0.02981, 154), (dt.date(2032,10,15), 0.0582, 0.02546, 256), (dt.date(2027,6,1), 0.0549, 0.0375, 205), (dt.date(2032,7,1), 0.0588, 0.02292, 256), 
#                    (dt.date(2029,7,1), 0.0607, 0.0224, 102), (dt.date(2031,5,15), 0.0564, 0.03479, 154), (dt.date(2024,7,1), 0.0569, 0.02637, 102), (dt.date(2024,8,1), 0.0574, 0.03605, 102), 
#                    (dt.date(2026,8,15), 0.0543, 0, 102)]}
# ladder = {'bonds':[(*b, relative_spread(b[0], b[1], 'tmuni')) for b in ladder['bonds']]}

simulate = GUI(model_params, portfolio, ladder, BH=False)

AppLayout(children=(VBox(children=(Label(value='Plotting Yield Curves:'), HBox(children=(ToggleButton(value=Tr…